#***MoodforMusic***
(An Intelligent Mood Detection and Music Recommendation
Application)

In [1]:
#import all necessary modules related to project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
#warnings
import warnings
warnings.filterwarnings('ignore')

#***Load the Dataset From Kaggle:-***

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
#api kaggle in kaggle cli
!kaggle datasets download rpjinu/moodformusic-dataset

Dataset URL: https://www.kaggle.com/datasets/rpjinu/moodformusic-dataset
License(s): MIT
 68% 41.0M/60.4M [00:00<00:00, 84.5MB/s]
100% 60.4M/60.4M [00:00<00:00, 107MB/s] 


#***Unzip the file:-***

In [4]:
#unzip the file
zip_ref = zipfile.ZipFile('/content/moodformusic-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
# Load CSV file
music_df = pd.read_csv('/content/data_moods.csv')

In [6]:
music_df.head()

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,1999,1999,Prince,2H7PHVdQ3mXqEHXcvclTB0,1982-10-27,68,379266,0.866,0.13700,0.730,0.000000,0.0843,0.625,-8.201,0.0767,118.523,5,4,Happy
1,23,23,Blonde Redhead,4HIwL9ii9CcXpTOTzMq0MP,2007-04-16,43,318800,0.381,0.01890,0.832,0.196000,0.1530,0.166,-5.069,0.0492,120.255,8,4,Sad
2,9 Crimes,9,Damien Rice,5GZEeowhvSieFDiR8fQ2im,2006-11-06,60,217946,0.346,0.91300,0.139,0.000077,0.0934,0.116,-15.326,0.0321,136.168,0,4,Sad
3,99 Luftballons,99 Luftballons,Nena,6HA97v4wEGQ5TUClRM0XLc,1984-08-21,2,233000,0.466,0.08900,0.438,0.000006,0.1130,0.587,-12.858,0.0608,193.100,4,4,Happy
4,A Boy Brushed Red Living In Black And White,They're Only Chasing Safety,Underoath,47IWLfIKOKhFnz1FUEUIkE,2004-01-01,60,268000,0.419,0.00171,0.932,0.000000,0.1370,0.445,-3.604,0.1060,169.881,1,4,Energetic


In [7]:
# Set paths for training and testing data
train = '/content/train'
test = '/content/test'

In [14]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              686 non-null    object 
 1   album             686 non-null    object 
 2   artist            686 non-null    object 
 3   id                686 non-null    object 
 4   release_date      686 non-null    object 
 5   popularity        686 non-null    int64  
 6   length            686 non-null    int64  
 7   danceability      686 non-null    float64
 8   acousticness      686 non-null    float64
 9   energy            686 non-null    float64
 10  instrumentalness  686 non-null    float64
 11  liveness          686 non-null    float64
 12  valence           686 non-null    float64
 13  loudness          686 non-null    float64
 14  speechiness       686 non-null    float64
 15  tempo             686 non-null    float64
 16  key               686 non-null    int64  
 1

In [21]:
#check the null values
music_df.isnull().sum().any()

False

In [22]:
#check dupliacted
music_df.duplicated().sum()

0

In [35]:
music_df['mood'].value_counts()

,count
mood,
Sad,197
Calm,195
Energetic,154
Happy,140


## ***Data Preprocessing:-***

In [8]:
img_width, img_height = 150, 150
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [9]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [10]:
# Building the Mood Classification Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])


In [11]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 189s 203ms/step - accuracy: 0.2332 - loss: 1.8500 - val_accuracy: 0.2604 - val_loss: 1.7670
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 189s 194ms/step - accuracy: 0.2598 - loss: 1.7867 - val_accuracy: 0.2811 - val_loss: 1.7402
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 181s 200ms/step - accuracy: 0.2673 - loss: 1.7626 - val_accuracy: 0.3011 - val_loss: 1.7174
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 176s 195ms/step - accuracy: 0.2723 - loss: 1.7608 - val_accuracy: 0.3086 - val_loss: 1.6911
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 201s 194ms/step - accuracy: 0.2863 - loss: 1.7377 - val_accuracy: 0.3468 - val_loss: 1.6451
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 202s 194ms/step - accuracy: 0.3004 - loss: 1.7158 - val_accuracy: 0.3622 - val_loss: 1.6493
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 199s 191ms/step - accuracy: 0.3193 - loss: 1.6836 - val_accuracy: 0.3991 - val_loss: 1.5615
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 203s 192ms/step - accuracy: 0.3355 -

In [13]:
# Evaluate the model
eval_result = model.evaluate(test_generator)
print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.4428 - loss: 1.4497
Test Loss: 1.433569073677063, Test Accuracy: 0.44747841358184814


In [17]:
# Generate Classification Report
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
y_pred = model.predict(test_generator, steps=test_steps_per_epoch)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step


In [18]:
print("Classification Report: \n", classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys()))

Classification Report: 
               precision    recall  f1-score   support

       angry       0.16      0.05      0.08       958
     disgust       0.00      0.00      0.00       111
        fear       0.13      0.07      0.09      1024
       happy       0.23      0.34      0.28      1774
     neutral       0.17      0.21      0.18      1233
         sad       0.17      0.17      0.17      1247
    surprise       0.11      0.13      0.12       831

    accuracy                           0.18      7178
   macro avg       0.14      0.14      0.13      7178
weighted avg       0.17      0.18      0.17      7178



In [19]:
# Save the model
model.save('mood_classification_model.h5')

#check out model working or not:_

In [24]:
# Music Recommendation Engine
mood_to_music = {
    'Happy': music_df[music_df['mood'] == 'Happy'],
    'Sad': music_df[music_df['mood'] == 'Sad'],
    'Surprise': music_df[music_df['mood'] == 'Surprise'],
    'Energetic': music_df[music_df['mood'] == 'Energetic']
}

In [31]:
def recommend_music(mood):
    recommendations = mood_to_music.get(mood, pd.DataFrame()).sample(n=1)
    for _, row in recommendations.iterrows():
        print(f"Recommended Music: {row['name']}, Album: {row['album']}, Artist: {row['artist']}")

In [34]:
# Example Usage
predicted_mood = 'Sad'  # This should come from the model prediction
recommend_music(predicted_mood)

Recommended Music: Throw Me a Bone, Album: Throw Me a Bone, Artist: Goat Girl
